In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import random
import csv

In [ ]:
def fetch_books(query, num_books=10):
    base_url = "https://openlibrary.org/search.json"
    response = requests.get(base_url, params={"q": query, "limit": num_books, "language": "eng"})

    if response.status_code == 200:
        data = response.json()
        books = []
        for book in data.get("docs", []):
            title = book.get("title")
            publication_year = book.get("first_publish_year")
            ls_isbn = book.get("isbn")
            ls_author = book.get("author_name")
            ls_lccn = book.get("lccn")
            ls_publicationplace = book.get("publish_place")
            ls_pages = book.get("number_of_pages_median")
            genre = query
            author = ls_author[0] if ls_author and isinstance(ls_author, list) else None
            isbn = ls_isbn[0] if ls_isbn and isinstance(ls_isbn, list) else None
            lccn = ls_lccn[0] if ls_lccn and isinstance(ls_lccn, list) else None
            place = ls_publicationplace[0] if ls_publicationplace and isinstance(ls_publicationplace, list) else None
            pages = ls_pages

            books.append({"title": title, "author": author, "isbn": isbn, "publicationyear": publication_year, "pages": pages,
                          "place": place, "lccn": lccn, "genre": genre, "callnumber": []})
        return books
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return []

ddc_map = {
    "Computer": "005",
    "Business": "330",
    "Engineering": "620",
    "Health": "610",
    "Law": "340",
    "Arts": "700",
    "Psychology": "150",
    "Design": "740",
    "Environmental": "333",
    "Education": "370"
}

def generate_call_number(book, ddc_map):
    """
    Generate a call number based on the book's genre and author's last name.
    """
    classification_number = ddc_map.get(book["genre"], "000")
    author_lastname = book["author"].split()[-1] if book["author"] else "UNK"

    author_code = author_lastname[:3].upper()  # Handles cases with less than 3 characters
    return f"{classification_number}.{author_code}"

def generate_periods_duplicatecount(periods, duplicate_count):
  periods = periods - duplicate_count
  return periods, duplicate_count

def duplicate_generation(duplicate_count, array):
  array_duplicates = np.random.choice(array, size=duplicate_count, replace=True)
  array = list(array) + list(array_duplicates)
  np.random.shuffle(array)
  return array

def default_generation(array, default_count, default_value):
    default_indices = np.random.choice(range(len(array)), size=default_count, replace=False)
    for idx in default_indices:
        array[idx] = default_value
    return array

def generate_semester_code(start_year):
    terms = {'Autumn': '09', 'Spring': '01', 'Summer': '04'}
    term = random.choice(list(terms.values()))
    year = random.randint(start_year, start_year + 4)  #maximum course period of 5 years
    return f"{term}{str(year)[2:]}"

def generate_price():
    return round(random.uniform(5, 1000), 2)

def generate_replacement_price(original_price):
    return round(original_price * random.uniform(1.0, 1.5), 2)

def assign_null_values(array, null_count):
    null_indices = np.random.choice(range(len(array)), size=null_count, replace=False)
    for idx in null_indices:
        array[idx] = None
    return array

def generate_course_code(level, department):
    year = random.randint(2001, 2024)  # Random year between 2001 and 2024
    return f"{level}{department}{year}"

def calculate_issn_check_digit(issn_7):
    total = 0
    for i in range(7):
        total += int(issn_7[i]) * (8 - i)
    remainder = total % 11
    return 'X' if remainder == 1 else str((11 - remainder) % 11)


In [ ]:
genres = [
    "Computer",
    "Business",
    "Engineering",
    "Health",
    "Law",
    "Arts",
    "Psychology",
    "Design",
    "Environmental",
    "Education"
]

title = []
author = []
pages = []
publicationyear = []
place = []
lccn = []
isbn = []
all_genres = []
callnumber = []

for genre in genres:
    book_details = fetch_books(genre, 100)
    if book_details:
        for book in book_details:
            title.append(book['title'])
            author.append(book['author'])
            pages.append(book['pages'])
            publicationyear.append(book['publicationyear'])
            place.append(book['place'])
            lccn.append(book['lccn'])
            isbn.append(book['isbn'])
            all_genres.append(book['genre'])
            book_callnumber = generate_call_number(book, ddc_map)
            book['callnumber'] = book_callnumber
            callnumber.append(book['callnumber'])

    else:
      print("No data fetched.")

In [ ]:
book_data = fetch_books("computer", 10)

if book_data:
    for i, book in enumerate(book_data):
        print(f"{i+1}. isbn: {book['isbn']}, publicationyear: {book['publicationyear']}, pages: {book['pages']}, lccn: {book['lccn']}, \
        place: {book['place']}")
else:
    print("No data fetched.")

1. isbn: 0451035801, publicationyear: 1968, pages: 237, lccn: 75362540,         place: Brasil
2. isbn: 9780521312097, publicationyear: 1505, pages: 431, lccn: 11032231,         place: Basileae
3. isbn: 1512405256, publicationyear: 1791, pages: 228, lccn: 17004948,         place: Berkeley, Calif
4. isbn: 0899670172, publicationyear: 1843, pages: 115, lccn: 2014031183,         place: New York, N.Y
5. isbn: 9780449205297, publicationyear: 1950, pages: 224, lccn: 63006943,         place: Greenwich, CT
6. isbn: 9788418318672, publicationyear: 1999, pages: 288, lccn: 2012044867,         place: Llandysul
7. isbn: None, publicationyear: 1987, pages: 30, lccn: 93126402,         place: Gaithersburg, MD (P.O. Box 6015, Gaithersburg 20877)
8. isbn: 9780256066982, publicationyear: 1986, pages: 144, lccn: 87082178,         place: None
9. isbn: 1427271283, publicationyear: 1998, pages: 447, lccn: 2006700040,         place: Beijing Shi
10. isbn: 9780471730279, publicationyear: 1999, pages: 728, lccn: 

In [ ]:
#BiblioFramwork Table

frameworkcode = ["ACQ", "AR", "BKS", "CF", "FA", "IR", "KT", "SER", "SR", "VR"]
frameworktext = [
    "Acquisition framework",
    "Models",
    "Books, Booklets, Workbooks",
    "CD-ROMs, DVD-ROMs, General Online Resources",
    "Fast Add Framework",
    "Binders",
    "Kits",
    "Serials",
    "Audio Cassettes, CDs",
    "DVDs, VHS"
]

biblio_framework = dict(zip(frameworkcode, frameworktext))

df_biblio_framework = pd.DataFrame(data=biblio_framework.items(), columns=['frameworkcode', 'frameworktext'])

df_biblio_framework.set_index('frameworkcode', inplace=True)

df_biblio_framework.to_csv('BiblioFramework.csv')

In [ ]:
#biblio table

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

np.random.seed(42)

n_points = 1000
frameworkcode_null_count = int(0.15 * 1000)
copyrightdate_null_count = int(0.10 * 1000)
copyrightdate_duplicate_count = int(0.2 * 1000)
author_null_count = int(0.05 * 1000)
title_null_count = int(0.05 * 1000)

biblionumber = list(range(1, 1001))
frameworkcode = np.random.choice(["ACQ", "AR", "BKS", "CF", "FA", "IR", "KT", "SER", "SR", "VR"], n_points)
random_indices = np.random.choice(biblionumber, frameworkcode_null_count, replace=False)
df_biblio_frameworkcode = pd.DataFrame([frameworkcode])
df_biblio_frameworkcode[random_indices] = None
biblio_frameworkcode = list(np.concatenate(df_biblio_frameworkcode.values))

timestamp = [datetime.now().strftime("%Y-%m-%d %H:%M:%S") for _ in range(1000)]

datecreated = pd.date_range(start="1990-01-01", end="2024-12-31", periods=1000)\
.strftime("%Y-%m-%d %H:%M:%S").tolist()

periods, duplicatecount = generate_periods_duplicatecount(1000, copyrightdate_duplicate_count )

copyrightdate = pd.date_range(start="1990-01-01", end="2024-12-31", periods=periods)\
.strftime("%Y-%m-%d %H:%M:%S").tolist()
copyrightdate = duplicate_generation(duplicatecount, copyrightdate)
copyrightdate = assign_null_values(copyrightdate, copyrightdate_null_count)
author = assign_null_values(author, author_null_count)
title = assign_null_values(title, title_null_count)


df_biblio = pd.DataFrame({
    'biblionumber': biblionumber,
    'biblio_frameworkcode': biblio_frameworkcode,
    'author': author,
    'title': title,
    'timestamp': timestamp,
    'datecreated': datecreated,
    'copyrightdate': copyrightdate
})

df_biblio.to_csv('Biblio.csv', index=False)


In [ ]:
#Biblioitems Table

np.random.seed(42)
pd.options.mode.copy_on_write = True



n_points = 1000
biblioitemnumber = ['biblioitems' + str(i) for i in range(1, 1001)]
itemtype = ["Books", "Computer Files", "Continuing Resources", "DVD", "DVD-Blueray", "Ebook", "Maps", "Mixed Materials", "Music", "Reference"]
itemtype = np.random.choice(itemtype, n_points).tolist()

itemtype_null_count = int(0.1 * 1000)
pages_null_count = int(0.15 * 1000)
isbn_null_count = int(0.05 * 1000)
lccn_null_count = int(0.1 * 1000)
issn_null_count = int(0.1 * 1000)
publicationyear_null_count = int(0.12 * 1000)
place_null_count = int(0.09 * 1000)


issn = []
for _ in range(1000):
    issn_7 = ''.join(str(random.randint(0, 9)) for _ in range(7))
    check_digit = calculate_issn_check_digit(issn_7)
    issn.append(issn_7 + check_digit)

#publicationyear = [int(x) if x is not None else 0 for x in publicationyear]

#biblionumber = default_generation(df_biblio['biblionumber'], 100, 0)
pages = assign_null_values(pages, pages_null_count)
isbn = assign_null_values(isbn, isbn_null_count)
lccn = assign_null_values(lccn, lccn_null_count)
publicationyear = assign_null_values(publicationyear, publicationyear_null_count)
place = assign_null_values(place, place_null_count)
itemtype = assign_null_values(itemtype, itemtype_null_count)
issn = assign_null_values(issn, issn_null_count)

df_biblioitems = pd.DataFrame({
    'biblioitemnumber': biblioitemnumber,
    'itemtype': itemtype,
    'pages': pd.Series(pages, dtype="Int64"),
    'isbn': isbn,
    'lccn': lccn,
    'issn': issn,
    'publicationyear': pd.Series(publicationyear, dtype="Int64"),
    'place': place,
    'biblionumber': biblionumber

})

df_biblioitems.to_csv('Biblioitems.csv', index=False)

In [ ]:
df_branches = pd.DataFrame({
    'branchname': [
        'College Lane LRC',
        'de Havilland LRC',
        'Online Library'
    ],
    'branchaddress1': [
        'College Lane',
        'de Havilland Campus',
        ''
    ],
    'branchaddress2': [
        'Hatfield, Hertfordshire',
        'Hatfield, Hertfordshire',
        ''
    ],
    'branchaddress3': [
        '',
        '',
        ''
    ],
    'branchzip': [
        'AL10 9AB',
        'AL10 9EU',
        ''
    ],
    'branchcity': [
        'Hatfield',
        'Hatfield',
        ''
    ],
    'branchstate': [
        'Hertfordshire',
        'Hertfordshire',
        ''
    ],
    'branchcountry': [
        'United Kingdom',
        'United Kingdom',
        ''
    ],
    'branchphone': [
        '+44 1707 284000',
        '+44 1707 284500',
        ''
    ],
    'branchemail': [
        'college.lrc@herts.ac.uk',
        'dehavilland.lrc@herts.ac.uk',
        'online.library@herts.ac.uk'
    ],
    'branchcode': [
        'CL-LRC',
        'DH-LRC',
        'ONLINE-LIB'
    ]
})


df_branches.to_csv('Branches.csv', index=False)
df_branches

,branchname,branchaddress1,branchaddress2,branchaddress3,branchzip,branchcity,branchstate,branchcountry,branchphone,branchemail,branchcode
0,College Lane LRC,College Lane,"Hatfield, Hertfordshire",,AL10 9AB,Hatfield,Hertfordshire,United Kingdom,+44 1707 284000,college.lrc@herts.ac.uk,CL-LRC
1,de Havilland LRC,de Havilland Campus,"Hatfield, Hertfordshire",,AL10 9EU,Hatfield,Hertfordshire,United Kingdom,+44 1707 284500,dehavilland.lrc@herts.ac.uk,DH-LRC
2,Online Library,,,,,,,,,online.library@herts.ac.uk,ONLINE-LIB


In [ ]:
#Itemtypes table

itemtypes_list = [
    {"itemtype": "AUDIO_RECORDING", "description": "Audio Recording"},
    {"itemtype": "BOOK_EBOOK", "description": "Book / eBook"},
    {"itemtype": "BOOK_CHAPTER", "description": "Book Chapter"},
    {"itemtype": "BOOK_REVIEW", "description": "Book Review"},
    {"itemtype": "CONFERENCE_PROCEEDING", "description": "Conference Proceeding"},
    {"itemtype": "DATA_SET", "description": "Data Set"},
    {"itemtype": "DISSERTATION_THESIS", "description": "Dissertation/Thesis"},
    {"itemtype": "DVD", "description": "DVD"},
    {"itemtype": "GOVERNMENT_DOCUMENT", "description": "Government Document"},
    {"itemtype": "JOURNAL_ARTICLE", "description": "Journal Article"},
    {"itemtype": "LIBRARY_HOLDING", "description": "Library Holding"},
    {"itemtype": "MAGAZINE_ARTICLE", "description": "Magazine Article"},
    {"itemtype": "MARKET_RESEARCH", "description": "Market Research"},
    {"itemtype": "NEWSLETTER", "description": "Newsletter"},
    {"itemtype": "NEWSPAPER_ARTICLE", "description": "Newspaper Article"},
    {"itemtype": "PAPER", "description": "Paper"},
    {"itemtype": "POSTER", "description": "Poster"},
    {"itemtype": "PRESENTATION", "description": "Presentation"},
    {"itemtype": "PUBLICATION", "description": "Publication"},
    {"itemtype": "PUBLICATION_ARTICLE", "description": "Publication Article"},
    {"itemtype": "REFERENCE", "description": "Reference"},
    {"itemtype": "REPORT", "description": "Report"},
    {"itemtype": "STANDARD", "description": "Standard"},
    {"itemtype": "STREAMING_AUDIO", "description": "Streaming Audio"},
    {"itemtype": "TEACHING_EXPERIENCE", "description": "Teaching Experience Collection"},
    {"itemtype": "TECHNICAL_REPORT", "description": "Technical Report"},
    {"itemtype": "TRADE_PUBLICATION_ARTICLE", "description": "Trade Publication Article"},
    {"itemtype": "TRANSCRIPT", "description": "Transcript"},
    {"itemtype": "VIDEO_STREAM", "description": "Videostream"},
    {"itemtype": "WEB_RESOURCE", "description": "Web Resource"}
]

df_itemtypes = pd.DataFrame(itemtypes_list)

df_itemtypes.to_csv('Itemtypes.csv', index=False)


In [ ]:
#Items table

np.random.seed(42)

price = [generate_price() for _ in range(1000)]
replacementprice = [generate_replacement_price(price) for price in price]

itemnumber =  ['item' + str(i) for i in range(1, 1001)]

items_biblionumber = biblionumber
item_biblioitemnumber = biblioitemnumber

item_biblioitemnumber = default_generation(item_biblioitemnumber, 100, 0)
items_biblionumber = default_generation(items_biblionumber, 150, 0)

barcode_null_count = int(0.1 * 1000)
dateaccessioned_null_count = int(0.12 * 1000)
booksellerid_null_count = int(0.2 * 1000)
homebranch_null_count = int(0.05 * 1000)
prices_null_count = int(0.2 * 1000)
replacementprice_null_count = int(0.10 * 1000)
replacementpricedate_null_count = int(0.2 * 1000)
datelastborrowed_null_count = int(0.2 * 1000)
date_lastseen_null_count = int(0.2 * 1000)
itemlost_null_count = int(0.05 * 1000)
withdrawn_null_count = int(0.2 * 1000)
itemcallnumber_null_count = int(0.05 * 1000)
holdingbranch_null_count = int(0.05 * 1000)
location_null_count = int(0.1 * 1000)
issues_null_count = int(0.1 * 1000)
renewals_null_count = int(0.1 * 1000)
reserves_null_count = int(0.1 * 1000)
restricted_null_count = int(0.1 * 1000)
itype_null_count = int(0.1 * 1000)
item_biblioitemnumber_null_count = int(0.05 * 1000)
items_biblionumber_null_count = int(0.05 * 1000)
onloan_null_count = int(0.5 * 1000)

barcode = [f"{i:010d}" for i in range(1000000000, 1000001000)]
dateaccessioned = pd.date_range(start="1990-01-01", end="2023-12-31", periods=1000)\
.strftime("%Y-%m-%d %H:%M:%S").tolist()

dt_dateaccessioned = pd.to_datetime(dateaccessioned)

#replacement price date
replacementpricedate_random_days = np.random.randint(30, 41, size=1000)
replacementpricedate = dt_dateaccessioned + pd.to_timedelta(replacementpricedate_random_days, unit = 'days')
replacementpricedate = replacementpricedate.strftime("%Y-%m-%d %H:%M:%S").tolist()

#date last borrowed
datelastborrowed_random_days = np.random.randint(20, 201, size=1000)
datelastborrowed = dt_dateaccessioned + pd.to_timedelta(datelastborrowed_random_days, unit = 'days')
datelastborrowed = datelastborrowed.strftime("%Y-%m-%d %H:%M:%S").tolist()

#onloan
onloan = pd.date_range(start="2024-01-01", end="2024-11-1", periods=1000)\
.strftime("%Y-%m-%d %H:%M:%S").tolist()


#date last seen
date_lastseen_random_days = np.random.randint(10, 101, size=1000)
date_lastseen = dt_dateaccessioned + pd.to_timedelta(date_lastseen_random_days, unit = 'days')
date_lastseen = date_lastseen.strftime("%Y-%m-%d %H:%M:%S").tolist()

bookseller_list = [
    "AMAZON-UK", "BOOK-DEPOSITORY", "BLACKWELLS", "WATERSTONES", "FOYLES",
    "PEARSON", "WILEY", "OXFORD-UPRESS", "CAMBRIDGE-UPRESS", "SPRINGER",
    "ELSEVIER", "INGRAM", "BAKER-TAYLOR", "FOLLETT", "THRIFTBOOKS",
    "ABEBOOKS", "LIBSUPPLY", "HERTS-BOOKSUPPLY"
]

booksellerid = [random.choice(bookseller_list) for _ in range(1000)]

shelving_locations = ["Main Shelves", "Reference", "Reserve", "Periodicals", "Archives", "Audiovisual", "Stacks", "Oversize", "Map Collection",
                      "Theses/Dissertations", "Children's Books",
                      "Non-fiction", "Fiction", "E-Books", "Special Collections", "Multimedia", "Closed Stacks", "Online Resources"]

location = [random.choice(shelving_locations) for _ in range(1000)]

homebranch = [random.choice(df_branches['branchcode']) for _ in range(1000)]
holdingbranch = [random.choice(df_branches['branchcode']) for _ in range(1000)]

issues_range = (0, 1000)  # For issues: 0 to 1000
renewals_range = (0, 10)  # For renewals: 0 to 10
reserves_range = (0, 50)  # For reserves: 0 to 50
restricted_range = (0, 1)  # For restricted: 0 (no restrictions), 1 (restricted)

issues = [random.randint(*issues_range) for _ in range(1000)]
renewals = [random.randint(*renewals_range) for _ in range(1000)]
reserves = [random.randint(*reserves_range) for _ in range(1000)]
restricted = [random.choice([0, 1]) for _ in range(1000)]



notforloan = [1] * 1000
damaged = [0] * 1000
withdrawn = [0] * 1000
itemlost = [0] * 1000
withdrawn = [0] * 1000

itemlost = default_generation(itemlost, 50, 1)
damaged = default_generation(damaged, 150, 1)
withdrawn = default_generation(withdrawn, 200, 1)

itemlost_on = []
withdrawn_on = []

#itemlost_on
for item in itemlost:
  if item == 1:
    itemlost_on_random_days = np.random.randint(5, 101, size=1000)
    itemlost_on = dt_dateaccessioned + pd.to_timedelta(itemlost_on_random_days, unit = 'days')
    itemlost_on = itemlost_on.strftime("%Y-%m-%d %H:%M:%S").tolist()

for i in withdrawn:
  if i == 1:
    withdrawn_on_random_days = np.random.randint(5, 101, size=1000)
    withdrawn_on = dt_dateaccessioned + pd.to_timedelta(withdrawn_on_random_days, unit = 'days')
    withdrawn_on = withdrawn_on.strftime("%Y-%m-%d %H:%M:%S").tolist()

itemcallnumber = callnumber

itype = [random.choice(df_itemtypes['itemtype']) for _ in range(1000)]


dateaccessioned = assign_null_values(dateaccessioned, dateaccessioned_null_count)
barcode = assign_null_values(barcode, barcode_null_count)
booksellerid = assign_null_values(booksellerid, booksellerid_null_count)
homebranch = assign_null_values(homebranch, homebranch_null_count)
price = assign_null_values(price, prices_null_count)
replacementprice = assign_null_values(replacementprice, replacementprice_null_count)
replacementpricedate = assign_null_values(replacementpricedate, replacementpricedate_null_count)
lastborrowed = assign_null_values(datelastborrowed, datelastborrowed_null_count)
date_lastseen = assign_null_values(date_lastseen, date_lastseen_null_count)
itemlost_on = assign_null_values(itemlost_on, itemlost_null_count)
withdrawn_on = assign_null_values(withdrawn_on, withdrawn_null_count)
itemcallnumber = assign_null_values(itemcallnumber, itemcallnumber_null_count)
holdingbranch = assign_null_values(holdingbranch, holdingbranch_null_count)
location = assign_null_values(location, location_null_count)
issues = assign_null_values(issues, issues_null_count)
renewals = assign_null_values(renewals, renewals_null_count)
reserves = assign_null_values(reserves, reserves_null_count)
restricted = assign_null_values(restricted, restricted_null_count)
itype = assign_null_values(itype, itype_null_count)
item_biblioitemnumber = assign_null_values(item_biblioitemnumber, item_biblioitemnumber_null_count)
items_biblionumber = assign_null_values(items_biblionumber, items_biblionumber_null_count)
onloan = assign_null_values(onloan, onloan_null_count)


df_items = pd.DataFrame({
    "itemnumber": itemnumber,
    "items_biblionumber": pd.Series(items_biblionumber, dtype="Int64"),
    "item_biblioitemnumber": item_biblioitemnumber,
    "barcode": barcode,
    "dateaccessioned": dateaccessioned,
    "booksellerid": booksellerid,
    "homebranch": homebranch,
    "holdingbranch": holdingbranch,
    "location": location,
    "price": price,
    "replacementprice": replacementprice,
    "replacementpricedate": replacementpricedate,
    "datelastborrowed": datelastborrowed,
    "date_lastseen": date_lastseen,
    "onloan": onloan,
    "issues": pd.Series(issues, dtype="Int64"),
    "renewals": pd.Series(renewals, dtype="Int64"),
    "reserves": pd.Series(reserves, dtype="Int64"),
    "restricted": pd.Series(restricted, dtype="Int64"),
    "notforloan": notforloan,
    "damaged": damaged,
    "withdrawn": withdrawn,
    "itemlost": itemlost,
    "itemlost_on": itemlost_on,
    "withdrawn_on": withdrawn_on,
    "itemcallnumber": itemcallnumber,
    "itype": itype
})

df_items.to_csv('Items.csv', index=False)

In [ ]:
#Categories Table

user_categories = [
    {"category_code": "STUDENT", "description": "Regular students"},
    {"category_code": "STAFF", "description": "Library staff"},
    {"category_code": "FACULTY", "description": "Faculty members"},
    {"category_code": "ALUMNI", "description": "Former students"},
    {"category_code": "GUEST", "description": "Temporary library users"},
    {"category_code": "RETIRED", "description": "Retired faculty/staff"},
    {"category_code": "EXTERNAL", "description": "External library users"},
    {"category_code": "PATRON", "description": "General public users"},
    {"category_code": "HIGH SCHOOL", "description": "High school students"},
    {"category_code": "GRADUATE", "description": "Graduate students"},
    {"category_code": "INSTRUCTOR", "description": "Instructors"},
    {"category_code": "NON-RESIDENT", "description": "Non-resident users"},
    {"category_code": "PUBLIC", "description": "General public"}
]

df_categories = pd.DataFrame(user_categories)

df_categories.to_csv('Categories.csv', index=False)

In [ ]:
#Borrower table

borrowernumber  =  ['borrower' + str(i) for i in range(1, 1001)]
cardnumber =  [str(random.randint(10**9, 10**10 - 1)) for _ in range(1000)]

df_names = pd.read_csv('name_data.csv')

# Extract the firstname and lastname columns
firstname = df_names['firstname'].tolist()
lastname = df_names['lastname'].tolist()
email = df_names['email'].tolist()

titles = [pd.NA] * 1000
othernames = [pd.NA] * 1000
initials = [pd.NA] * 1000
streetnumber = [pd.NA] * 1000
streettype = [pd.NA] * 1000
address = [pd.NA] * 1000
address2 = [pd.NA] * 1000
city = [pd.NA] * 1000
state = [pd.NA] * 1000
zipcode = [pd.NA] * 1000
country = [pd.NA] * 1000
email = [pd.NA] * 1000
phone = [pd.NA] * 1000
mobile = [pd.NA] * 1000

B_streetnumber = [pd.NA] * 1000
B_streettype = [pd.NA] * 1000
B_address = [pd.NA] * 1000
B_address2 = [pd.NA] * 1000
B_city = [pd.NA] * 1000
B_state = [pd.NA] * 1000
B_zipcode = [None] * 1000
B_country = [None] * 1000
B_email = [None] * 1000
B_phone = [None] * 1000
dateofbirth = [None] * 1000
altcontactfirstname = [None] * 1000
altcontactsurname = [None] * 1000
altcontactaddress1 = [None] * 1000
altcontactaddress2 = [None] * 1000
altcontactaddress3 = [None] * 1000
altcontactstate = [None] * 1000
altcontactzipcode = [None] * 1000
altcontactcountry = [None] * 1000
altcontactphone = [None] * 1000
lost_null_count = int(0.1 * 1000)
debarred_null_count = int(0.1 * 1000)
userid_null_count = int(0.1 * 1000)

userid = [random.randint(10000000, 99999999) for _ in range(1000)]

branchcode = [random.choice(df_branches['branchcode']) for _ in range(1000)]
date_enrolled = pd.date_range(start="1990-01-01", end="2024-12-31", periods=1000)\
.strftime("%Y-%m-%d %H:%M:%S").tolist()

debarred_random_days = np.random.randint(30, 41, size=1000)
debarred = dt_dateaccessioned + pd.to_timedelta(replacementpricedate_random_days, unit = 'days')
debarred = debarred.strftime("%Y-%m-%d %H:%M:%S").tolist()

category = [random.choice(df_categories['category_code']) for _ in range(1000)]

date_expiry = pd.date_range(start="2025-01-01", end="2030-12-31", periods=1000)\
.strftime("%Y-%m-%d %H:%M:%S").tolist()

lost = [0] * 1000
lost = default_generation(lost, 50, 1)


lost = assign_null_values(lost, lost_null_count)
debarred = assign_null_values(debarred, debarred_null_count)
userid = assign_null_values(userid, userid_null_count)
borrowers_data = {
    'borrowernumber': borrowernumber,
    'cardnumber': cardnumber,
    'firstname': firstname,
    'lastname': lastname,
    'email': email,
    'titles': titles,
    'othernames': othernames,
    'initials': initials,
    'streetnumber': streetnumber,
    'streettype': streettype,
    'address': address,
    'address2': address2,
    'city': city,
    'state': state,
    'zipcode': zipcode,
    'country': country,
    'phone': phone,
    'mobile': mobile,
    'B_streetnumber': B_streetnumber,
    'B_streettype': B_streettype,
    'B_address': B_address,
    'B_address2': B_address2,
    'B_city': B_city,
    'B_state': B_state,
    'B_zipcode': B_zipcode,
    'B_country': B_country,
    'B_email': B_email,
    'B_phone': B_phone,
    'dateofbirth': dateofbirth,
    'altcontactfirstname': altcontactfirstname,
    'altcontactsurname': altcontactsurname,
    'altcontactaddress1': altcontactaddress1,
    'altcontactaddress2': altcontactaddress2,
    'altcontactaddress3': altcontactaddress3,
    'altcontactstate': altcontactstate,
    'altcontactzipcode': altcontactzipcode,
    'altcontactcountry': altcontactcountry,
    'altcontactphone': altcontactphone,
    'lost': pd.Series(lost, dtype="Int64"),
    'debarred': debarred,
    'userid': userid,
    'branchcode': branchcode,
    'date_enrolled': date_enrolled,
    'date_expiry': date_expiry,
    'category': category
}

df_borrowers = pd.DataFrame(borrowers_data)
df_borrowers.to_csv('Borrowers.csv', index=False)


In [ ]:
#Courses table

courses = pd.read_csv('course_data.csv')

course_id = ['course' + str(i) for i in range(1, courses.shape[0] + 1)]
coursename = courses['Course Name'].tolist()
coursecode = courses['Course Code'].tolist()
department = courses['Department'].tolist()


section = [generate_semester_code(random.randint(2021, 2024)) for _ in range(courses.shape[0])]
students_count = np.random.randint(100, 2000 + 1, size=courses.shape[0]).tolist()

enabled = ['yes'] * courses.shape[0]
enabled = default_generation(enabled, 8, 'no')

coursename_null_count = int(0.03 * courses.shape[0])
coursecode_null_count = int(0.03 * courses.shape[0])
department_null_count = int(0.03 * courses.shape[0])
section_null_count = int(0.05 * courses.shape[0])
students_count_null_count = int(0.03 * courses.shape[0])

coursename = assign_null_values(coursename, coursename_null_count)
coursecode = assign_null_values(coursecode, coursecode_null_count)
department = assign_null_values(department, department_null_count)
section = assign_null_values(section, section_null_count)
students_count = assign_null_values(students_count, students_count_null_count)

courses_data = {
    'course_id': course_id,
    'coursecode': coursecode,
    'coursename': coursename,
    'department': department,
    'section': section,
    'students_count': pd.Series(students_count, dtype="Int64"),
    'enabled': enabled
}

df_courses = pd.DataFrame(courses_data)
df_courses.to_csv('Courses.csv', index=False)

In [ ]:
#Issues table

issue_id  =  ['issue' + str(i) for i in range(1, 1001)]

borrowernumber_null_count = int(0.1 * 1000)
itemnumber_null_count = int(0.1 * 1000)
branchcode_null_count = int(0.1 * 1000)
timestamp_null_count = int(0.1 * 1000)
issuedate_null_count = int(0.1 * 1000)
date_due_null_count = int(0.1 * 1000)
lastreneweddate_null_count = int(0.1 * 1000)
renewals_null_count = int(0.1 * 1000)


borrowernumber = [random.choice(df_borrowers['borrowernumber']) for _ in range(1000)]

itemnumber = [random.choice(df_items['itemnumber']) for _ in range(1000)]

timestamp = [datetime.now().strftime("%Y-%m-%d %H:%M:%S") for _ in range(1000)]

auto_renew = [0] * 1000
auto_renew = default_generation(lost, 500, 1)

branchcode = [random.choice(df_branches['branchcode']) for _ in range(1000)]

datelastborrowed = pd.date_range(start="2024-01-01", end="2024-10-01", periods=1000)

periods, duplicate_count = generate_periods_duplicatecount(1000, 200)

issuedate_range = pd.date_range(start="2023-01-01", end="2024-02-01", periods=periods)

issuedate_random_days = np.random.randint(10, 101, size=periods)

issuedate = issuedate_range -  pd.to_timedelta(issuedate_random_days, unit = 'days')

issuedate = issuedate.strftime("%Y-%m-%d %H:%M:%S").tolist()

issuedate = duplicate_generation(duplicate_count, issuedate)

date_due_random_days = np.random.randint(14, size=1000) #two-week loan
date_due = datelastborrowed + pd.to_timedelta(date_due_random_days, unit = 'days')
date_due = date_due.strftime("%Y-%m-%d %H:%M:%S").tolist()

returndate = []

issuedate = assign_null_values(issuedate, issuedate_null_count)


for issued in issuedate:
  if issued != None:
    returndate = pd.NA
  else:
    returndate_random_days = np.random.randint(0, 101, size=1000)
    returndate = datelastborrowed - pd.to_timedelta(returndate_random_days, unit = 'days')
    returndate = returndate.strftime("%Y-%m-%d %H:%M:%S").tolist()

lastreneweddate = []
renewals = []
renewals_range = (0, 10)

for renew in auto_renew:
  if renew == 1:
    lastreneweddate_random_days = np.random.randint(0, 30, size=1000)
    lastreneweddate = datelastborrowed + pd.to_timedelta(lastreneweddate_random_days, unit = 'days')
    lastreneweddate = lastreneweddate.strftime("%Y-%m-%d %H:%M:%S").tolist()

    renewals = [random.randint(*renewals_range) for _ in range(1000)]


borrowernumber = assign_null_values(borrowernumber, borrowernumber_null_count)
itemnumber = assign_null_values(itemnumber, itemnumber_null_count)
branchcode = assign_null_values(branchcode, branchcode_null_count)
timestamp = assign_null_values(timestamp, timestamp_null_count)
issuedate = assign_null_values(issuedate, issuedate_null_count)
date_due = assign_null_values(date_due, date_due_null_count)
lastreneweddate = assign_null_values(lastreneweddate, lastreneweddate_null_count)
renewals = assign_null_values(renewals, renewals_null_count)


df_issues = pd.DataFrame({
    'issue_id': issue_id,
    'borrowernumber': borrowernumber,
    'itemnumber': itemnumber,
    'branchcode': branchcode,
    'timestamp': timestamp,
    'issuedate': issuedate,
    'date_due': date_due,
    'lastreneweddate': lastreneweddate,
    'renewals': pd.Series(renewals, dtype="Int64")
})

df_issues.to_csv('Issues.csv', index=False)


In [ ]:
#course_items

ci_id = ['ci' + str(i) for i in range(1, 1001)]
course_id = [random.choice(df_items["itemnumber"]) for _ in range(1000)]
holdingbranch = [random.choice(df_branches['branchcode']) for _ in range(1000)]

holdingbranch_null_count = int(0.1 * 1000)

holdingbranch = assign_null_values(holdingbranch, holdingbranch_null_count)

enabled = ['yes'] * 1000
enabled = default_generation(enabled, 8, 'no')

df_courseitems = pd.DataFrame({
    'ci_id': ci_id,
    'course_id': course_id,
    'holdingbranch': holdingbranch,
    'enabled': enabled
})

df_courseitems.to_csv('course_items.csv', index=False)


In [ ]:
#Coursereserves table

cr_id = ['cr' + str(i) for i in range(1, 1001)]
course_id = [random.choice(df_courses['coursecode']) for _ in range(1000)]
ci_id = [random.choice(df_courseitems['ci_id']) for _ in range(1000)]

df_coursereserves = pd.DataFrame({
    'cr_id': cr_id,
    'course_id': course_id,
    'ci_id': ci_id
})

df_coursereserves.set_index('cr_id', inplace=True)

df_coursereserves.to_csv('Coursereserves.csv')

In [ ]:
#course_instructors table
course_id = df_courses['course_id']

borrowernumber = []


borrowernumber = df_borrowers[df_borrowers['category'] == 'INSTRUCTOR']['borrowernumber']
course_id = [random.choice(df_courses['course_id']) for _ in range(len(borrowernumber))]




df_course_instructors = pd.DataFrame({
    'course_id': course_id,
    'borrowernumber': borrowernumber
})

df_course_instructors = df_course_instructors.dropna(subset=['borrowernumber'])

df_course_instructors.set_index(['course_id', 'borrowernumber'], inplace=True)

df_course_instructors.to_csv('course_instructors.csv')